In [93]:
#library imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import warnings
from matplotlib.pyplot import figure
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression

In [4]:
#data imports
returns = pd.read_pickle("./iX/differential-capital/Data/returns.pkl")
volumes = pd.read_csv('./iX/differential-capital/Data/stock_volumes.csv', index_col = 'date', parse_dates = True)
info = pd.read_csv('./iX/differential-capital/Data/stock_info.csv', index_col = 'Instrument')
sp_listings = pd.read_csv('./iX/differential-capital/Data/sp500_listings.csv', index_col = 'date', parse_dates = True)

### Pipeline psuedo code
    for t in range (first_date to last_date):
        find stocks in investable universe at t (stocks in the S&P500 that have prices  
                                                 recorded for the last n days)
    
        for stock in investable universe:
            get training data
            fit the model
            predict return for t+1
        
        #how much return we should make according to predictions
        top5 predicted returns at time t+1
        bottom5 predicted returns at time t+1
        predicted average return = top5 - bottom 5
        
        #how much return we actually make
        top5 actual returns at time t+1 =
        bottom5 actual returns at time t+1 = 
        actual average return = top5 - bottom5

In [5]:
#drop very first row
returns = returns.iloc[1:, :]

In [98]:
def get_investable(t, n_rows):
    "Find stocks in investable universe at time t\
    (stocks in the S&P500 that have prices recorded for the last n_rows days)"
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    most_recent = t
    least_recent = t + pd.DateOffset(-n_rows)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable[most_recent+pd.DateOffset(-1):least_recent]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n_rows].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    print(len(investable_universe))
    
    return df_investable

In [106]:
def get_training(df_investable, stock, t):
    stock = stock
    time = pd.to_datetime(t)
    
    #save the target (sample stock's returns upto time t)
    y = df_investable[[stock]]
    y.columns = [str(stock) + ' t+1']
    y = y.sort_index(ascending = False)
    
    #shift the features
    X = df_investable.loc[(time + pd.DateOffset(-1)):, :]
    X = X.sort_index(ascending = False)

    #concatenate outcome and features
    train = pd.concat([X, y], axis = 1)
    train = train.sort_index(ascending = False)
    
    #shit outcome down 1 row
    train[str(stock) + ' t+1'] = train[str(stock) + ' t+1'].shift(1)
    train = train.iloc[1:, :]
    
    #split into X_train and y_train
    X_train = train.drop(str(stock) + ' t+1', axis = 1)
    y_train = train[[str(stock) + ' t+1']]
    
    return X_train, y_train

In [ ]:
def get_test(stock, t):
    

In [94]:
def fit_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    print('Fitted model')

In [ ]:
def pred_next(X_test, y_test):
    predictions = model.predict(X_test)
    df_eval = pd.concat([X_test, y_test], axis = 1)
    df_eval['predictions'] = predictions
    return df_eval

In [107]:
time = pd.to_datetime('1999-02-24')
investable = get_investable(time, 80)
investable.head()

848


,905270,921795,904261,905261,916328,936365,902355,912215,905271,921246,...,906828,545918,545588,325895,923298,992765,922853,311917,543755,131745
date,,,,,,,,,,,,,,,,,,,,,
1999-02-23,-0.576744,-1.551363,-0.504323,-0.718177,-0.132424,-2.913124,-0.444444,-1.667515,-2.025804,-1.684533,...,-1.345756,1.139070,-3.687083,0.946970,-0.727277,0.000000,0.000000,0.926204,-0.745092,4.413908
1999-02-22,0.111753,5.019815,4.282494,0.937383,2.717539,9.575035,0.106781,2.025974,2.259972,1.240310,...,1.594155,-3.824320,0.231131,1.734104,-0.182359,-1.298701,29.166667,-1.818717,1.512727,-0.743955
1999-02-19,0.111878,0.000000,0.452830,-0.137294,0.000000,2.919708,0.106895,-1.521934,0.188414,0.000000,...,0.688310,-0.544770,-0.687799,3.592814,-1.076628,-1.910828,-7.692308,-3.509769,-2.225699,-2.143579
1999-02-18,1.668246,-3.155650,0.913938,1.843142,2.222244,0.735294,0.000000,0.000000,-4.158921,0.467290,...,3.073388,0.547754,-0.456985,4.375000,-0.713015,0.000000,0.000000,2.703492,3.847290,-0.702811
1999-02-17,0.228007,0.903614,-1.868460,-0.367275,-4.127726,-0.366884,0.000000,-2.335748,2.979519,-2.134146,...,0.000000,-3.683453,-4.782513,-1.234568,-1.232676,-1.875000,0.000000,-1.770419,0.000000,-0.697906


In [109]:
x_train, y_train = get_training(investable, '905270', time)
x_train.head()
#why is this starting at 1999-02-22 and not 1999-02-23?

,905270,921795,904261,905261,916328,936365,902355,912215,905271,921246,...,906828,545918,545588,325895,923298,992765,922853,311917,543755,131745
date,,,,,,,,,,,,,,,,,,,,,
1999-02-22,0.111753,5.019815,4.282494,0.937383,2.717539,9.575035,0.106781,2.025974,2.259972,1.240310,...,1.594155,-3.824320,0.231131,1.734104,-0.182359,-1.298701,29.166667,-1.818717,1.512727,-0.743955
1999-02-19,0.111878,0.000000,0.452830,-0.137294,0.000000,2.919708,0.106895,-1.521934,0.188414,0.000000,...,0.688310,-0.544770,-0.687799,3.592814,-1.076628,-1.910828,-7.692308,-3.509769,-2.225699,-2.143579
1999-02-18,1.668246,-3.155650,0.913938,1.843142,2.222244,0.735294,0.000000,0.000000,-4.158921,0.467290,...,3.073388,0.547754,-0.456985,4.375000,-0.713015,0.000000,0.000000,2.703492,3.847290,-0.702811
1999-02-17,0.228007,0.903614,-1.868460,-0.367275,-4.127726,-0.366884,0.000000,-2.335748,2.979519,-2.134146,...,0.000000,-3.683453,-4.782513,-1.234568,-1.232676,-1.875000,0.000000,-1.770419,0.000000,-0.697906
1999-02-16,0.611738,-0.257511,-3.602305,-1.447828,4.305443,-2.846959,0.339650,2.641026,-2.007321,0.923077,...,1.682292,0.000000,-0.217093,1.039501,-0.350318,0.000000,-4.878049,0.446380,0.000000,-9.558161


In [110]:
y_train.head()
#why is this starting at 1999-02-22 and not 1999-02-23?

,905270 t+1
date,
1999-02-22,-0.576744
1999-02-19,0.111753
1999-02-18,0.111878
1999-02-17,1.668246
1999-02-16,0.228007
